# 1 Анализ тарифов:
* «Мегалайн» — федеральный оператор сотовой связи. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра». Необходимо понять, какой тариф приносит больше денег.
* Необходимо сделать предварительный анализ тарифов на небольшой выборке клиентов. В нашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.
* Описание тарифов
   1. Тариф «Смарт»
     Ежемесячная плата: 550 рублей
     Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
     Стоимость услуг сверх тарифного пакета: 1. минута разговора: 3 рубля («Мегалайн» всегда округляет вверх значения минут и        мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута); 2. сообщение: 3 рубля; 3.      1 Гб интернет-трафика: 200 рублей.
   2.  Тариф «Ультра»
     Ежемесячная плата: 1950 рублей
     Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
     Стоимость услуг сверх тарифного пакета: 1. минута разговора: 1 рубль; 2. сообщение: 1 рубль; 3. 1 Гб интернет-трафика: 150      рублей.

**План работы:**

    1. Изучение файлов и общей информации
    2. Подготовка данных
    3. Анализ данных
    4. Проверка гипотез
    5. Общий вывод


**Примечание**:
  «Мегалайн» всегда округляет секунды до минут, а мегабайты — до гигабайт. Каждый звонок округляется отдельно: даже если он       длился всего 1 секунду, будет засчитан как 1 минута.
  Для веб-трафика отдельные сессии не считаются. Вместо этого общая сумма за месяц округляется в бо́льшую сторону. Если абонент   использует 1025 мегабайт в этом месяце, с него возьмут плату за 2 гигабайта.

## Изучение данных из файла


In [1]:
#Импортируем библиотеки
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats as st
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Изучим таблицу users (информация о пользователях)
try:
    data_users = pd.read_csv('***')  # Локальный путь
except:
    data_users = pd.read_csv('***') # Серверный путь 
data_users.info() #общая информация о данных
print(data_users.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in data_users.columns:
    print("-----------------------------")
    print(col)
    display(data_users[col].describe(include=None))
    display(data_users[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

print(data_users.isna().sum()*100/len(data_users)) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', data_users.duplicated().sum()) #проверяем количество дубликатов

**Вывод**

 * В таблице users всего 500 строк. В столбце churn_date 92,4% пропущенных значений, но это означает, что тариф ещё действовал на момент выгрузки данных
 * Тип столбцов churn_date — дата прекращения пользования тарифом, city — город проживания пользователя, first_name — имя пользователя, last_name — фамилия пользователя, reg_date — дата подключения тарифа (день, месяц, год), tariff — название тарифного плана: текстовые или смешанные числовые и нечисловые значения;  user_id — уникальный идентификатор пользователя, age — возраст пользователя (годы): целочисленные значения.


In [ ]:
#Изучим таблицу calls
try:
    data_calls = pd.read_csv('C:/Users/datasets/calls.csv')  # Локальный путь
except:
    data_calls = pd.read_csv('/datasets/calls.csv') # Серверный путь 
data_calls.info() #общая информация о данных
print(data_calls.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in data_calls.columns:
    print("-----------------------------")
    print(col)
    display(data_calls[col].describe(include=None))
    display(data_calls[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

print(data_calls.isna().sum()*100/len(data_calls)) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', data_calls.duplicated().sum()) #проверяем количество дубликатов

**Вывод**

* В таблице calls всего 202607 строк и нет пропущенных значений.
Тип столбцов id — уникальный номер звонка и call_date — дата звонка: текстовые или смешанные числовые и нечисловые значения; duration — длительность звонка в минутах: числа с плавающей точкой; user_id — идентификатор пользователя, сделавшего звонок: целочисленные значения. Столбец duration необходимо будет округлить в большую сторону на след.этапе. Также можно заметить что в столбце duration есть нулевые значения - так обозначены пропущенные звонки.

In [ ]:
#Изучим таблицу internet
try:
    data_internet = pd.read_csv('C:/Users/datasets/internet.csv', index_col=[0])  # Локальный путь
except:
    data_internet = pd.read_csv('/datasets/internet.csv', index_col=[0]) # Серверный путь 
# Unnamed: 0 - Это не нужный столбец, удалили его командой index_col=[0] при чтении файла.
data_internet.info() #общая информация о данных
print(data_internet.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in data_internet.columns:
    print("-----------------------------")
    print(col)
    display(data_internet[col].describe(include=None))
    display(data_internet[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

print(data_internet.isna().sum()*100/len(data_internet)) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', data_internet.duplicated().sum()) #проверяем количество дубликатов

**Вывод**

* В таблице internet всего 149396 строк и нет пропущенных значений.
Тип столбцов id — уникальный номер сессии и session_date — дата интернет-сессии: текстовые или смешанные числовые и нечисловые значения; mb_used — объём потраченного за сессию интернет-трафика (в мегабайтах): числа с плавающей точкой; user_id — идентификатор пользователя: целочисленные значения. 

In [ ]:
#Изучим таблицу messages
try:
    data_messages = pd.read_csv('C:/Users/datasets/messages.csv')  # Локальный путь
except:
    data_messages = pd.read_csv('/datasets/messages.csv') # Серверный путь 
data_messages.info() #общая информация о данных
print(data_messages.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in data_messages.columns:
    print("-----------------------------")
    print(col)
    display(data_messages[col].describe(include=None))
    display(data_messages[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

print(data_messages.isna().sum()*100/len(data_messages)) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', data_messages.duplicated().sum()) #проверяем количество дубликатов

**Вывод**

* В таблице messages всего 123036 строк и нет пропущенных значений.
Типs столбцов id — уникальный номер сообщения и message_date — дата сообщения: текстовые или смешанные числовые и нечисловые значения; user_id — идентификатор пользователя, отправившего сообщение: целочисленные значения. 

In [ ]:
#Изучим таблицу tariffs
try:
    data_tariffs = pd.read_csv('C:/Users/datasets/tariffs.csv')  # Локальный путь
except:
    data_tariffs = pd.read_csv('/datasets/tariffs.csv') # Серверный путь 
data_tariffs.info() #общая информация о данных
print(data_tariffs.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in data_tariffs.columns:
    print("-----------------------------")
    print(col)
    display(data_tariffs[col].describe(include=None))
    display(data_tariffs[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

data_tariffs.isna().sum()*100/len(data_tariffs) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', data_tariffs.duplicated().sum()) #проверяем количество дубликатов

**Вывод**

* В таблице internet всего 2 строки и нет пропущенных значений. В ней описаны тарифные планы и что в них входит (условия использования).
Тип столбцов tariff_name — название тарифа — дата интернет-сессии: текстовые или смешанные числовые и нечисловые значения; остальные столбцы: целочисленные значения. 

## Подготовка данных


In [ ]:
#Округлим данные в столбце duration из таблицы calls в большую сторону
data_calls['duration'] = data_calls['duration'].apply(np.ceil).astype(int)

#Объединим имя и фамилию клиента в один столбец
data_users['full_name']= data_users['first_name'] + ' ' + data_users['last_name']

#Выделим столбцы с месяцами в каждой таблице
data_users['month']= pd.to_datetime(data_users['reg_date'])
data_users['month'] = data_users['month'].dt.month_name()

data_calls['month']= pd.to_datetime(data_calls['call_date'])
data_calls['month'] = data_calls['month'].dt.month_name()

data_messages['month']= pd.to_datetime(data_messages['message_date'])
data_messages['month'] = data_messages['month'].dt.month_name()

data_internet['month']= pd.to_datetime(data_internet['session_date'])
data_internet['month'] = data_internet['month'].dt.month_name()

#Проработаем чило пропусков в 'churn_date', заменим пустые значения на дату выгрузки
data_users.loc[data_users['churn_date'].isna(), 'churn_date'] = datetime.date(2018, 12, 31)

#Проверим: дата оттока всегда позже или равна дате регистрации
data_users['values_date'] = pd.to_datetime(data_users['churn_date']) >= pd.to_datetime(data_users['reg_date'])
data_users['values_date'].value_counts()


**Вывод**

* Округлили столбец с минутами соответственно исходным данным («Мегалайн» каждый звонок округляется отдельно: даже если он длился всего 1 секунду, будет засчитан как 1 минута).

* Для удобства объеднили в таблице users столбец с Именем и Фамилией клиента.

* Также для упрощения исследования данных по месяцам - во всех таблицах выделили необходимые столбцы с месяцами.

* Пропущенные значения в столбце churn_date означают, что тариф ещё действовал на момент выгрузки данных. Для удобства работы с данным столбцом заменили все пропущенные значения на дату выгрузки, а именно 2018-12-31.



In [ ]:
#Посчитаем для каждого пользователя количество сделанных звонков и израсходованных минут разговора по месяцам

data_calls['month'] = pd.Categorical(data_calls['month'], ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"])

calls_users = data_calls.pivot_table(index = ['user_id', 'month'], values = 'duration', aggfunc = [ 'count', 'sum'])
calls_users.columns = ['count_duration', 'sum_duration']
#display(calls_users)

calls_users.head(20) #Посмотрим 20 первых значений



In [ ]:
#Количество отправленных сообщений по месяцам
data_messages['month'] = pd.Categorical(data_messages['month'], ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"])

messages_users = data_messages.pivot_table(index = ['user_id', 'month'], values = 'id', aggfunc = 'count').astype(int)
messages_users.columns = ['count_messages']
display(messages_users)


In [ ]:
#Объем израсходованного интернет-трафика по месяцам
data_internet['month'] = pd.Categorical(data_internet['month'], ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"])

internet_users = data_internet.pivot_table(index = ['user_id', 'month'], values = 'mb_used', aggfunc = 'sum')
internet_users.columns = ['sum_mb_used']
#Добавим столбец с Гб, применив условие задания: общая сумма веб-трафика за месяц округляется в бо́льшую сторону
internet_users['sum_gb_used'] = (internet_users['sum_mb_used']/1024).apply(np.ceil).astype(int)
display(internet_users)

In [ ]:
#Объединим таблицы с помощью Merge:
users_month_info = calls_users.merge(messages_users, on = ['user_id', 'month'], how = 'outer').merge(internet_users, on = ['user_id', 'month'], how = 'outer').reset_index()
data = users_month_info.merge(data_users[['user_id','tariff', 'city']], on='user_id', how = 'outer')
data.rename(columns = {'tariff' : 'tariff_name'}, inplace = True)
data = data.merge(data_tariffs, on = 'tariff_name', how = 'outer')
display(data)



In [ ]:
#Посчитаем помесячную выручку с каждого пользователя:
#Вычтим бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; 
#Остаток умножим на значение из тарифного плана; прибавим абонентскую плату, соответствующую тарифному плану).
data['minutes_over'] =  data['sum_duration'] - data['minutes_included'] 
data['minutes_over'] = np.where(data['minutes_over'] > 0, data['minutes_over'], 0) # там, где пользователи не истратили лимит по тарифу, поставим значение 0
data['minutes_over_rub'] = data['minutes_over']*data['rub_per_minute']
 
data['messages_over'] =  data['count_messages'] - data['messages_included'] 
data['messages_over'] = np.where(data['messages_over'] > 0, data['messages_over'], 0) # там, где пользователи не истратили лимит по тарифу, поставим значение 0
data['messages_over_rub'] = data['messages_over']*data['rub_per_message']
 
data['gb_used_over'] =  data['sum_gb_used'] - data['mb_per_month_included']/1024
data['gb_used_over'] = np.where(data['gb_used_over'] > 0, data['gb_used_over'], 0) # там, где пользователи не истратили лимит по тарифу, поставим значение 0
data['gb_used_over_rub'] = data['gb_used_over']*data['rub_per_gb']

data['total_revenue'] = data['minutes_over_rub'] + data['messages_over_rub'] + data['gb_used_over_rub'] + data['rub_monthly_fee'] # выручка

display(data)


#Проведем категоризацию данных по городу
def city_group(row):
    if row['city'] == 'Москва':
        return 'Москва'
    return 'другой'
data['city_group'] = data.apply(city_group, axis=1)
print(data)


dfs = dict(tuple(data.groupby('tariff_name'))) # разделим таблицу по тарифа


#Построим гистрограмму, чтобы посмотреть выручку по тарифам 
dfs['ultra']['total_revenue'].hist(bins=100, range=(1900,4000), figsize=(10,5))
plt.title('Гистограмма по выручке тариф ультра')
plt.xlabel('total_revenue')
plt.ylabel('values')
plt.show()


data.info()


In [ ]:
dfs['smart']['total_revenue'].hist(bins=100, range=(400,4500), figsize=(10,5))
plt.title('Гистограмма по выручке тариф смарт')
plt.xlabel('total_revenue')
plt.ylabel('values')
plt.show()

#Построим диаграммы размаха для каждого тарифа по выручке
#Тариф snmart
plt.figure(figsize=(10,5))
dfs['smart'].boxplot(column =['total_revenue'])
plt.title('Диаграмма размаха smart')
plt.ylabel('values')
plt.show()

#Тариф ultra
plt.figure(figsize=(10,5))
dfs['ultra'].boxplot(column =['total_revenue'])
plt.title('Диаграмма размаха ultra')
plt.ylabel('values')
plt.show()


In [ ]:
print(dfs['ultra']['total_revenue'].describe())
print(dfs['smart']['total_revenue'].describe())

**Вывод**

* Мы посчитайте для каждого пользователя:
количество сделанных звонков и израсходованных минут разговора по месяцам;
количество отправленных сообщений по месяцам;
объем израсходованного интернет-трафика по месяцам.
Объединили все необходимые данные в одну таблицу (название и описание тарифа).
И в итоге мы посчитали помесячную выручку с каждого пользователя (столбец total_revenue)

* По гистограммам видно, что в тарифе ультра в основном пользователи не выходят за рамки тарифа, в отличае от тарифа смарт.

* По диаграммам размаха видно, что медиана у тарифа smart меньше, чем тарифа ultra, но датасет скошен вправо. Этим тарифом пользуются те, кто платит ощутимо больше медианной суммы. Если клиенты готовы платить больше, возможно им стоит предложить подключать сразу тариф ultra.


## Анализ данных

Опишем поведение клиентов оператора, исходя из выборки. Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц. Посчитаем среднее количество, дисперсию и стандартное отклонение. Построим гистограммы. Опишем распределения.

In [ ]:
data_new = data.pivot_table(index='tariff_name', values = ['sum_duration','count_messages', 'sum_gb_used' ], aggfunc = ['mean', 'var', 'std']).round(1).reset_index()
data_new.columns = ['tariff_name', 'mean_count_messages', 'mean_sum_duration', 'mean_sum_gb_used', 'var_count_messages', 'var_sum_duration', 'var_sum_gb_used','std_count_messages', 'std_sum_duration', 'std_sum_gb_used' ]
display(data_new)

In [ ]:
print('Дисперсия использованных минут, тариф smart:', data_new.loc[0,'var_sum_duration'])
print('Стандартное отклонение использованных минут, тариф smart:', data_new.loc[0,'std_sum_duration'])
print('Среднее количество потраченных минут, тариф smart:', data_new.loc[0,'mean_sum_duration'])
print('---------------------------------------------------------------------')
print('Дисперсия использованных минут, тариф ultra:', data_new.loc[1,'var_sum_duration'])
print('Стандартное отклонение использованных минут, тариф ultra:', data_new.loc[1,'std_sum_duration'])
print('Среднее количество потраченных минут, тариф ultra:', data_new.loc[1,'mean_sum_duration'])
print('---------------------------------------------------------------------')

#Построим гистограмму
plt.figure(figsize=(15,8))
plt.hist((dfs['smart']['sum_duration'], dfs['ultra']['sum_duration']), bins=120, alpha=0.5, label=['smart', 'ultra'], color=['r', 'b'])
plt.legend(loc='upper right')
plt.title('Гистограмма по количеству потраченных минут')
plt.xlabel('sum_duration')
plt.ylabel('values')
plt.show()


In [ ]:
print('Дисперсия количества сообщений, тариф smart:', data_new.loc[0,'var_count_messages'])
print('Стандартное отклонение количества сообщений, тариф smart:',  data_new.loc[0,'std_count_messages'])
print('Среднее количество потраченных сообщений, тариф smart:',  data_new.loc[0,'mean_count_messages'])
print('---------------------------------------------------------------------')
print('Дисперсия количества сообщений, тариф ultra:', data_new.loc[1,'var_count_messages'])
print('Стандартное отклонение количества сообщений, тариф ultra:',  data_new.loc[1,'std_count_messages'])
print('Среднее количество потраченных сообщений, тариф ultra:',  data_new.loc[1,'mean_count_messages'])
print('---------------------------------------------------------------------')

#Построим гистограмму
plt.figure(figsize=(15,8))
plt.hist((dfs['smart']['count_messages'], dfs['ultra']['count_messages']), bins=120, alpha=0.5,  label=['smart', 'ultra'], color=['r', 'b'])
plt.legend(loc='upper right')
plt.title('Гистограмма по количеству потраченных сообщений')
plt.xlabel('count_messages')
plt.ylabel('values')
plt.show()


In [ ]:
print('Дисперсия количества интернет-трафика, тариф smart:', data_new.loc[0,'var_sum_gb_used'])
print('Стандартное отклонение интернет-трафика, тариф smart:', data_new.loc[0,'std_sum_gb_used'])
print('Среднее количество интернет-трафика, тариф smart:', data_new.loc[0,'mean_sum_gb_used'])
print('---------------------------------------------------------------------')
print('Дисперсия количества интернет-трафика, тариф ultra:', data_new.loc[1,'var_sum_gb_used'])
print('Стандартное отклонение интернет-трафика, тариф ultra:', data_new.loc[1,'std_sum_gb_used'])
print('Среднее количество интернет-трафика, тариф ultra:', data_new.loc[1,'mean_sum_gb_used'])
print('---------------------------------------------------------------------')

plt.figure(figsize=(15,8))
plt.hist((dfs['smart']['sum_gb_used'], dfs['ultra']['sum_gb_used']), bins=100, alpha=0.5,  label=['smart', 'ultra'], color=['r', 'b'])
plt.legend(loc='upper right')
plt.title('Гистограмма по количеству потраченного интернет-трафика')
plt.xlabel('sum_gb_used')
plt.ylabel('values')
plt.show()

**Вывод**

* Мы посчитали среднее количество, дисперсию и стандартное отклонение по количеству израсходованных минут, сообщений и интернет-трафика для каждого тарифа.

* Дисперсия показывает насколько данные сильно 'размазаны' относительно среднего. В данном случае видно, что у распределения длительности звонков, количества сообщений и интернет-трафика у тарифа ultra 'хвост' длинее, чем у тарифа smart. Это также видно на построеннных гистограммах.
* По графикам видно, что данные имеют нормальное распределение и по которым можно сделать выводы:
   + -пользоатели тарифа smart в основном тратят на разговоры  250 до 600 мин, а пользователи тарифа ultra от 0 до 1000 мин
   + -пользоатели тарифа smart используют сообщения менее активно (в основном до 50смс), чем пользователи тарифа ultra (в основном до 150смс)
   + -пик использованного трафика пользоателями тарифа smart приходится на 17-18Гб, а пользователи тарифа ultra - 20Гб и такого ярковыраженного пика не наблюдается.




## Проверка гипотез: 
Средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются: 
* Проверим нулевую гипотезу, что средняя выручка пользователей тарифов ultra и smart равна; альтернативная гипотеза - средняя выручка отличается

Средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов:
* Проверим нулевую гипотезу, что средняя выручка пользователей из Москвы равна выручке пользователей из других регионов; альтернативная гипотеза - средняя выручка отличается


In [ ]:
#Проверим нулевую гипотезу, что средняя выручка пользователей тарифов ultra и smart равна
alpha = .01 # критический уровень статистической значимости
# если p-value окажется меньше него - отвегнем гипотезу

results = st.ttest_ind(
    dfs['smart']['total_revenue'], 
    dfs['ultra']['total_revenue'], equal_var = False) # используем equal_var = False, так как выборки разного размера и дисперсии отличаются

print('p-значение: ', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

**Вывод**
* Средняя выручка зависит от тарифа.

In [ ]:
dfs_city = dict(tuple(data.groupby('city_group'))) # разделим таблицу по группе городов: Москва/другой
print(data['city_group'].value_counts())
print( dfs_city['Москва']['total_revenue'].describe())
print( dfs_city['другой']['total_revenue'].describe())

#Проверим нулевую гипотезу, что средняя выручка пользователей из Москвы равна выручке пользователей из других регионов
alpha = .01 # критический уровень статистической значимости
# если p-value окажется меньше него - отвегнем гипотезу

results = st.ttest_ind(
    dfs_city['Москва']['total_revenue'], 
    dfs_city['другой']['total_revenue'], equal_var = False)

print('p-значение: ', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 



**Вывод**
* Полученное значение p-value говорит о том, что вероятность получить такое или большее различие случайно почти 52%. Это явно слишком большая вероятность, чтобы делать вывод о значимом различии между средними выручками.

## Общий вывод

* Проанализировав данные можно сказать, что клиенты, пользующиеся тарифом Smart очень часто выходят за его рамки по сообщениям, звонкам и интернет-трафику. А пользователи Ultra не используют даже половину доступных в тарифе минут и сообщений, но иногда докупают интернет-трафик.

* Средняя выручка у тарифа smart меньше, чем тарифа ultra, но датасет скошен вправо. Этим тарифом пользуются те, кто платит ощутимо больше медианной суммы. Ешё  можем предположить, что средняя выручка тарифов различна, а вот средняя выручка между пользователями Москвы и других городов не имеет значительной разницы.

<span style="color:gren">***Я предложила бы заказчику обратить внимание, что тариф Ultra более перспективен, им пользует меньшее количество пользователей, но он приносит больше выручки. А тем, кто много переплачивает в тарифе smart предложить перейти на тариф Ultra. Так будет выгоднее и для заказчика, и для пользователей*** </span>.

 
